In [14]:

import pandas as pd
import numpy as np
import re
import string

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('french'))
import unicodedata



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nmako\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
data = {
    "avis": [
        "Le service client est excellent, je suis très satisfait",
        "Le taux d'intérêt est trop élevé",
        "J'adore cette banque, toujours rapide et efficace",
        "Le site internet est lent et plante souvent",
        "Merci pour votre réactivité lors de ma demande de prêt",
        "Je suis déçu, le conseiller n’a pas répondu à mes emails",
        "Application mobile claire et facile à utiliser",
        "Aucune transparence sur les frais, très frustrant",
        "Je recommande cette banque à mes amis",
        "Temps d’attente interminable au guichet",
        "Les conseillers sont aimables et compétents",
        "Je trouve vos services acceptables mais sans plus",
        "La banque offre des taux compétitifs",
        "Les frais cachés sont inacceptables",
        "Service en ligne très pratique et rapide",
        "J’ai eu un problème avec ma carte, mais le support a été efficace",
        "Les horaires d’ouverture ne sont pas adaptés",
        "L’accueil en agence est toujours chaleureux",
        "Les délais de traitement sont trop longs",
        "Bonne expérience globale avec cette banque",
        "Le site web est difficile à naviguer",
        "Les conseils financiers sont pertinents",
        "Je n’ai pas reçu mes relevés à temps",
        "Les distributeurs automatiques fonctionnent mal",
        "J’apprécie les offres personnalisées",
        "Les taux d’intérêt sur les prêts sont trop élevés",
        "L’application mobile plante fréquemment",
        "Le personnel est très professionnel",
        "Manque de communication lors des changements de contrat",
        "Les opérations en ligne sont sécurisées",
        "Les frais de tenue de compte sont excessifs",
        "Le processus de souscription est simple",
        "Les informations sur les produits sont claires",
        "J’ai eu des difficultés à joindre le service client",
        "Les offres promotionnelles sont intéressantes",
        "Les délais de remboursement sont raisonnables",
        "Je suis satisfait de la gestion de mon compte",
        "Les notifications ne fonctionnent pas correctement",
        "La banque propose une bonne gamme de produits",
        "Le conseiller est toujours disponible et à l’écoute",
        "Les services additionnels sont utiles",
        "Je ne recommande pas cette banque à cause des frais",
        "L’espace client en ligne est intuitif",
        "Le service après-vente est décevant",
        "J’ai obtenu un prêt facilement",
        "Les conditions des prêts sont avantageuses",
        "L’assistance téléphonique est efficace",
        "Le site manque de mises à jour régulières",
        "Les charges sont bien expliquées",
        "Je suis mécontent du service client",
        "La banque protège bien mes données personnelles",
        "Les rendez-vous en agence sont faciles à prendre"
    ],
    "sentiment": [
        "positif", "negatif", "positif", "negatif", "positif",
        "negatif", "positif", "negatif", "positif", "negatif",
        "positif", "neutre", "positif", "negatif", "positif",
        "positif", "negatif", "positif", "negatif", "positif",
        "negatif", "positif", "negatif", "negatif", "positif",
        "negatif", "negatif", "positif", "negatif", "positif",
        "negatif", "positif", "positif", "negatif", "positif",
        "positif", "positif", "negatif", "positif", "positif",
        "positif", "negatif", "positif", "negatif", "positif",
        "positif", "positif", "positif", "negatif", "positif",
        "negatif", "positif"
    ]
}

df = pd.DataFrame(data)

df

,avis,sentiment
0,"Le service client est excellent, je suis très ...",positif
1,Le taux d'intérêt est trop élevé,negatif
2,"J'adore cette banque, toujours rapide et efficace",positif
3,Le site internet est lent et plante souvent,negatif
4,Merci pour votre réactivité lors de ma demande...,positif
5,"Je suis déçu, le conseiller n’a pas répondu à ...",negatif
6,Application mobile claire et facile à utiliser,positif
7,"Aucune transparence sur les frais, très frustrant",negatif
8,Je recommande cette banque à mes amis,positif
9,Temps d’attente interminable au guichet,negatif


In [30]:
def nettoyer_texte(text):
    text = text.lower()  # minuscule
    text = re.sub(r'\d+', '', text)  # supprimer chiffres
    text = text.translate(str.maketrans('', '', string.punctuation))  # supprimer ponctuation
    text = " ".join([word for word in text.split() if word not in stop_words])  # stopwords
    text = ''.join(ch for ch in text if ch not in string.punctuation)
    text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode('utf-8')  # normalisation unicode
    text = re.sub(r"[^a-z\s]", "", text) #Supprimer espaces multiples
    return text

df["avis_nettoye"] = df["avis"].apply(nettoyer_texte)
df

,avis,sentiment,avis_nettoye
0,"Le service client est excellent, je suis très ...",positif,service client excellent tres satisfait
1,Le taux d'intérêt est trop élevé,negatif,taux dinteret trop eleve
2,"J'adore cette banque, toujours rapide et efficace",positif,jadore cette banque toujours rapide efficace
3,Le site internet est lent et plante souvent,negatif,site internet lent plante souvent
4,Merci pour votre réactivité lors de ma demande...,positif,merci reactivite lors demande pret
5,"Je suis déçu, le conseiller n’a pas répondu à ...",negatif,decu conseiller na repondu emails
6,Application mobile claire et facile à utiliser,positif,application mobile claire facile utiliser
7,"Aucune transparence sur les frais, très frustrant",negatif,aucune transparence frais tres frustrant
8,Je recommande cette banque à mes amis,positif,recommande cette banque amis
9,Temps d’attente interminable au guichet,negatif,temps dattente interminable guichet


In [32]:
# Séparation Train/Test
X_train, X_test, y_train, y_test = train_test_split(
    df["avis_nettoye"], df["sentiment"], test_size=0.3, random_state=42)

In [33]:
 #Vectorisation TF-IDF transformée les textes en nombres

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [41]:
#create a pipeline for the model
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('model', LogisticRegression())
])

In [44]:
#Model training
pipeline.fit(X_train, y_train)

,steps,"[('tfidf', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [45]:
#enregistre le modèle
import joblib
joblib.dump(pipeline, '../models/sentiment_model.pkl')

['../models/sentiment_model.pkl']

In [48]:
#utilisation du modèle pour prédire les sentiments sur les données de test
y_pred = pipeline.predict(X_test) 
# Affichage des résultats
print( y_pred)


['positif' 'positif' 'negatif' 'positif' 'positif' 'positif' 'positif'
 'positif' 'positif' 'positif' 'negatif' 'positif' 'positif' 'positif'
 'positif' 'positif']


In [58]:
test_texts = [
    "Je suis très content du service client.",
    "C'est une expérience horrible, je suis déçu.",
    "Le produit est correct, rien d'exceptionnel.",
]

preds = pipeline.predict(test_texts)
print(preds)


['positif' 'positif' 'positif']
